#### Testing Numpy


In [1]:
import numpy as np

In [2]:
data = np.array(
    [[1,1,1],
     [5,4,2],
     [9,3,4],
     [5,4,4]])

taus = np.array(
    [[0.5, 0.5],
     [0.5, 0.5],
     [0.5, 0.5],
     [0.5, 0.5]])

mus = np.array(
    [[0, 0, 0],
     [1, 1, 1]])

pis = np.array(
    [0.4, 0.6]
)

sigma = np.diag(np.ones(4))

In [3]:
m, n = data.shape
print(f'm: {m}')
print(f'n: {n}')

print(sigma)


m: 4
n: 3
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


#### GOOD

In [5]:
pis_next = np.sum(taus, axis=0)/m

mus_next = ((taus.T @ data).T/np.sum(taus, axis=0)).T

C = np.zeros(shape=(2,3,3))
for k in range(2):     # clusters
    for i in range(4): # datapoints
        tau_ik = taus[i,k]
        vec = data[i,:] - mus_next[k,:]
        C[k,:,:] += np.outer(vec, vec)*tau_ik
    C[k,:,:] /= np.sum(taus, axis=0)[k]

In [98]:
C[1]

array([[8.    , 2.    , 3.    ],
       [2.    , 1.5   , 1.    ],
       [3.    , 1.    , 1.6875]])

In [93]:
print(f'data: \n {data} \n')
print(f'mus_next: \n {mus_next} \n')
print(f'taus: \n {taus} \n')

print('----- For one cluster ----- \n')

print(f'x^i - mu_k: \n {data-mus_next[0]} \n')
print(f'tau_k: \n {taus[:,0]} \n')
print(f':Sum tau_k^i \n {np.sum(taus, axis=0)[0]} \n')
print(f'Sum tau_k^i * (x^i - mu_k): \n {(data-mus_next[0])*np.sum(taus, axis=0)[0]} \n')

# print(f'Sigma : \n {(((data-mus_next[0])*np.sum(taus, axis=0)[0]) @ ((data-mus_next[0]).T))/np.sum(taus, axis=0)[0]} \n') ### Wrong!

data: 
 [[1 1 1]
 [2 2 2]
 [3 3 3]
 [4 4 4]] 

mus_next: 
 [[2.5 2.5 2.5]
 [2.5 2.5 2.5]] 

taus: 
 [[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]] 

----- For one cluster ----- 

x^i - mu_k: 
 [[-1.5 -1.5 -1.5]
 [-0.5 -0.5 -0.5]
 [ 0.5  0.5  0.5]
 [ 1.5  1.5  1.5]] 

tau_k: 
 [0.5 0.5 0.5 0.5] 

:Sum tau_k^i 
 2.0 

Sum tau_k^i * (x^i - mu_k): 
 [[-3. -3. -3.]
 [-1. -1. -1.]
 [ 1.  1.  1.]
 [ 3.  3.  3.]] 



In [101]:
def fast_gaussian_pdf(x, mu, sigma, r):
    n, m = x.shape

    # Compute the eigendecomposition of sigma, in DESCENDING order of eigenvalues
    eVals, eVecs = np.linalg.eig(sigma)
    idx = eVals.argsort()[::-1]   
    eVals = eVals[idx]
    eVecs = eVecs[:,idx]

    # Compute the low-rank approximation of sigma
    U_hat = eVecs[:,0:r]
    Lambda_hat = np.diag(eVals[0:r])
    Sigma_hat = U_hat @ Lambda_hat @ U_hat.T

    # Compute the transformation of the data and parameters
    data_hat = (U_hat.T @ data.T).T
    mu_hat = (U_hat.T @ mu)

    # Compute the log-likelihood of the density
    A = -m*np.log(2*np.pi) - np.sum(np.log(eVals))
    B = -1/2*np.sum((data_hat-mu_hat)/eVals,axis=1)

    log_lik = np.exp(A+B)
    
    # Return the actual likelihood
    return np.exp(log_lik)

4 
 3 
 [[[8.     2.     3.    ]
  [2.     1.5    1.    ]
  [3.     1.     1.6875]]

 [[8.     2.     3.    ]
  [2.     1.5    1.    ]
  [3.     1.     1.6875]]]


In [26]:
r = 3
sigma = C[1]
mu = mus[1]

# Compute the eigendecomposition of sigma, in DESCENDING order of eigenvalues
eVals, eVecs = np.linalg.eig(sigma)
idx = eVals.argsort()[::-1]   
eVals = eVals[idx]
eVecs = eVecs[:,idx]

# Compute the low-rank approximation of sigma
U_hat = eVecs[:,0:r]
Lambda_hat = np.diag(eVals[0:r])
Sigma_hat = U_hat @ Lambda_hat @ U_hat.T

# Compute the transformation of the data and parameters
data_hat = (U_hat.T @ data.T).T
mu_hat = (U_hat.T @ mu)

# Compute the log-likelihood of the density
# A = -m*np.log(2*np.pi) - np.sum(np.log(eVals))
# B = -1/2*np.sum((data_hat-mu_hat)**2/eVals[0:r],axis=1)

log_lik = np.exp(A+B)

In [39]:
data_hat[2]

array([-10.28483599,   0.43505189,   0.18132429])

In [32]:
mu_hat

array([-1.51744272,  0.78171745,  0.29374382])

In [36]:
x_minus_mu = (data_hat[1] - mu_hat)

In [38]:
np.dot(x_minus_mu, x_minus_mu)

26.000000000000004

In [46]:
xmm = (data_hat - mu_hat)
xmm

array([[ 0.        ,  0.        ,  0.        ],
       [-4.72112217,  1.51602184, -1.1885635 ],
       [-8.76739328, -0.34666556, -0.11241954],
       [-5.44723659,  1.98751147,  0.61433844]])

In [55]:
print(f'x_hat: {data_hat[1]} \n')
print(f'mu_hat: {mu_hat} \n')
print(f'lambda_i: {eVals[0:r]} \n')

x_hat: [-6.23856489  2.29773929 -0.89481967] 

mu_hat: [-1.51744272  0.78171745  0.29374382] 

lambda_i: [9.79697239 0.9693119  0.42121571] 



In [59]:
xmm**2/eVals[0:r]

array([[0.        , 0.        , 0.        ],
       [2.27509007, 2.37108638, 3.35382355],
       [7.84601424, 0.12398177, 0.03000399],
       [3.02873023, 4.07526398, 0.89600579]])

In [23]:
# (data_hat - mu_hat) @ (data_hat - mu_hat)
                         
(data_hat[0] - mu_hat[0]).T @ (data_hat[0] - mu_hat[0])

8.56653414157383

In [25]:
# np.dot(data_hat - mu_hat, data_hat - mu_hat)

ValueError: shapes (4,3) and (4,3) not aligned: 3 (dim 1) != 4 (dim 0)

In [12]:
(data_hat-mu_hat)

array([[ 0.        ,  0.        ,  0.        ],
       [-4.72112217,  1.51602184, -1.1885635 ],
       [-8.76739328, -0.34666556, -0.11241954],
       [-5.44723659,  1.98751147,  0.61433844]])

In [7]:
log_lik

array([1.60405973e-04, 3.82802555e-04, 3.42896654e-04, 3.66440798e-05])

In [8]:
################ OLD ################

# Compute the log-likelihood of the density
A = (np.sum(np.log(eVals[0:r]))) ### Removed this since all probabilities were being rounded to 0
B = 0
for i in range(r):
    B += (x_hat[i] - mu_hat[i])**2/eVals[i]
log_lik = -(B/2) - A

################ OLD ################

NameError: name 'x_hat' is not defined

In [93]:
log_lik

array([0.00100786, 0.00240522, 0.00215448, 0.00023024])

In [89]:
print(f'data_hat: \n {data_hat} \n')
print(f'mu_hat: \n {mu_hat} \n')
print(f'data_hat - mu_hat: \n {data_hat - mu_hat} \n')
print(f'eVals: \n {eVals} \n')
print(f'(data_hat - mu_hat)/eVals: \n {(data_hat - mu_hat)/eVals} \n')


data_hat: 
 [[ -1.51744272   0.78171745   0.29374382]
 [ -6.23856489   2.29773929  -0.89481967]
 [-10.28483599   0.43505189   0.18132429]
 [ -6.9646793    2.76922892   0.90808226]] 

mu_hat: 
 [-1.51744272  0.78171745  0.29374382] 

data_hat - mu_hat: 
 [[ 0.          0.          0.        ]
 [-4.72112217  1.51602184 -1.1885635 ]
 [-8.76739328 -0.34666556 -0.11241954]
 [-5.44723659  1.98751147  0.61433844]] 

eVals: 
 [9.79697239 0.9693119  0.42121571] 

(data_hat - mu_hat)/eVals: 
 [[ 0.          0.          0.        ]
 [-0.48189604  1.56401861 -2.82174538]
 [-0.89490844 -0.35764088 -0.26689302]
 [-0.55601224  2.05043545  1.45848889]] 



In [90]:
np.sum((data_hat - mu_hat)/eVals,axis=1)

array([ 0.        , -1.73962281, -1.51944234,  2.9529121 ])

In [91]:
-0.48189604 + 1.56401861 -2.82174538

-1.73962281

In [82]:
np.array([[9.7969, 0.9693119, 0.42121571],
          [125,125,125],
          [0,0,0],
          [1,1,1]])/eVals

array([[9.99992611e-01, 1.00000000e+00, 9.99999990e-01],
       [1.27590438e+01, 1.28957460e+02, 2.96760059e+02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.02072351e-01, 1.03165968e+00, 2.37408047e+00]])